In [3]:
# ------------------- IMPORT LIBRARIES -------------------
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from tqdm import tqdm
import os

In [4]:
# ------------------- SETTINGS -------------------
data_path = "DATA/[P1] AIS Data/nari_dynamic.csv"  # AIS data file
master_csv = "near_collision_master.csv"           # Master CSV to store results
distance_threshold = 1852                          # meters (1 nautical mile)
chunk_size = 100_000                               # number of rows per chunk

In [5]:
# ------------------- FUNCTION TO CALCULATE DISTANCE -------------------
def compute_distance(row1, row2):
    """Return distance (meters) between two points (lat, lon)."""
    return geodesic((row1["lat"], row1["lon"]), (row2["lat"], row2["lon"])).meters


In [6]:
# ------------------- INITIALIZE MASTER CSV -------------------
master_csv = "near_collision_master.csv"
file_exists = os.path.exists(master_csv)  # check if file already exists

# ------------------- PROCESS DATA IN CHUNKS -------------------
chunk_counter = 0

for chunk in pd.read_csv(data_path, chunksize=chunk_size):
    chunk_counter += 1
    print(f"\nProcessing chunk {chunk_counter} with {len(chunk)} rows")

    # Rename column if needed
    if "sourcemmsi" in chunk.columns:
        chunk = chunk.rename(columns={"sourcemmsi": "mmsi"})

    # Drop missing crucial values
    chunk = chunk.dropna(subset=["mmsi", "lat", "lon", "t"])

    # Convert timestamp and sort
    chunk["datetime"] = pd.to_datetime(chunk["t"], unit="s")
    chunk = chunk.sort_values("datetime")

    # ------------------- FILTER MOVING SHIPS -------------------
    chunk = chunk[chunk["speedoverground"] > 0.5]  # only ships moving > 0.5 knots

    # ------------------- GROUP BY 1-MINUTE TIME WINDOWS -------------------
    chunk["time_window"] = chunk["datetime"].dt.floor("1min")
    grouped = chunk.groupby("time_window")

    # ------------------- NEAR-COLLISION DETECTION -------------------
    near_collisions = []

    for time_window, group in tqdm(grouped, desc=f"Chunk {chunk_counter}"):
        ships = group.to_dict("records")
        n = len(ships)
        
        seen_pairs = set()  # reset for each minute

        for i in range(n):
            for j in range(i + 1, n):
                ship1, ship2 = ships[i], ships[j]

                # Skip self-collision
                if ship1["mmsi"] == ship2["mmsi"]:
                    continue

                # Avoid duplicate pairs
                pair_id = tuple(sorted([ship1["mmsi"], ship2["mmsi"]]))
                if pair_id in seen_pairs:
                    continue
                seen_pairs.add(pair_id)

                # Compute distance
                dist = compute_distance(ship1, ship2)
                if dist < distance_threshold:
                    near_collisions.append({
                        "time_window": time_window,
                        "mmsi_1": ship1["mmsi"],
                        "mmsi_2": ship2["mmsi"],
                        "lat_1": ship1["lat"], "lon_1": ship1["lon"],
                        "lat_2": ship2["lat"], "lon_2": ship2["lon"],
                        "distance_m": dist,
                        "speed_1": ship1["speedoverground"], "speed_2": ship2["speedoverground"],
                        "course_1": ship1["courseoverground"], "course_2": ship2["courseoverground"]
                    })

    # ------------------- APPEND RESULTS TO MASTER CSV -------------------
    collisions_df = pd.DataFrame(near_collisions)
    print(f"Near-collision events found in chunk {chunk_counter}: {len(collisions_df)}")

    # Append to CSV with header only if file does not exist yet
    collisions_df.to_csv(master_csv, index=False, mode="a", header=not file_exists)

    # After first write, file now exists
    file_exists = True



Processing chunk 1 with 100000 rows


Chunk 1: 100%|██████████| 1929/1929 [00:10<00:00, 191.46it/s]


Near-collision events found in chunk 1: 1648

Processing chunk 2 with 100000 rows


Chunk 2: 100%|██████████| 1728/1728 [00:05<00:00, 300.09it/s]


Near-collision events found in chunk 2: 922

Processing chunk 3 with 100000 rows


...

Near-collision events found in chunk 189: 48600

Processing chunk 190 with 100000 rows


Chunk 190: 100%|██████████| 10222/10222 [00:40<00:00, 254.75it/s]


Near-collision events found in chunk 190: 50229

Processing chunk 191 with 35630 rows


Chunk 191: 100%|██████████| 4166/4166 [00:05<00:00, 793.49it/s] 

Near-collision events found in chunk 191: 1134


In [7]:
master_csv = "near_collision_master.csv"

# Load the master CSV
df_master = pd.read_csv(master_csv)

# Check total number of near-collision events
print("Total near-collision events found:", len(df_master))
print(df_master.head())


Total near-collision events found: 677044
           time_window     mmsi_1     mmsi_2      lat_1     lon_1      lat_2  \
0  2015-09-30 22:18:00  228854000  228931000  48.118610 -4.415707  48.121580   
1  2015-09-30 22:19:00  228133000  228854000  48.117535 -4.424553  48.118410   
2  2015-09-30 22:20:00  228854000  228133000  48.118250 -4.424275  48.117233   
3  2015-09-30 22:21:00  228133000  228854000  48.116974 -4.433182  48.118060   
4  2015-09-30 22:24:00  228854000  228133000  48.117516 -4.441568  48.116100   

      lon_2  distance_m  speed_1  speed_2  course_1  course_2  
0 -4.419288  424.450297     10.5      8.9     274.6      78.5  
1 -4.420028  350.704813     10.6     10.4     263.1     269.3  
2 -4.429567  409.924222     10.4     10.7     261.4     264.9  
3 -4.428580  363.300903     10.7     10.4     264.9     268.4  
4 -4.446808  420.718312     10.8     10.7     261.5     264.3  
